In [1]:
import pandas as pd 
import numpy as np
from os.path import dirname, join
import os
#pd.set_option("max_rows", None)

In [2]:
#function to call merged_gw.csv file in every season folder
def import_merged_gw(season='2020-21'):
    path = os.getcwd()
    filename = 'merged_gw.csv'
    season_path = join(dirname(dirname("__file__")), path, season, 'gws', filename)
    
    return season_path

In [3]:
df_1617 = pd.read_csv(import_merged_gw(season='2016-17'), encoding='latin-1')
df_1718 = pd.read_csv(import_merged_gw(season='2017-18'), encoding='latin-1')
df_1819 = pd.read_csv(import_merged_gw(season='2018-19'), encoding='latin-1')
df_1920 = pd.read_csv(import_merged_gw(season='2019-20'), encoding='utf-8')
df_2021 = pd.read_csv(import_merged_gw(), encoding='utf-8')

In [4]:
#add column season to every df
list_df = [df_1617, df_1718, df_1819, df_1920, df_2021]
seasons = ['2016-17', '2017-18', '2018-19', '2019-20', '2020-21']

for i, j in zip(list_df, seasons):
    i['season'] = j

#append all seasons df into one master df and filter columns
df = pd.concat([df_1617, df_1718, df_1819, df_1920, df_2021], ignore_index=True, sort=False)
df = df[['season','name', 'position', 'team', 'assists','bonus','bps','clean_sheets','creativity','element','fixture','goals_conceded','goals_scored','ict_index','influence','kickoff_time','minutes','opponent_team','own_goals','penalties_missed','penalties_saved','red_cards','round','saves','selected','team_a_score','team_h_score','threat','total_points','transfers_balance','transfers_in','transfers_out','value','was_home','yellow_cards','GW']]    

In [5]:
#name pattern: 
# 1617 & 1718 - Abdoulaye_Doucouré
# 1819 & 1920 - Abdoulaye_Doucouré_390
# 2021 - Abdoulaye Doucouré - final name to display

#replace _ with space in name column
new_df = df.copy()
new_df['name'] = new_df['name'].str.replace('_', ' ')
#remove number in name column
new_df['name'] = new_df['name'].str.replace('\d+', '')
#trim name column
new_df['name'] = new_df['name'].str.strip()

In [6]:
#fill in null position (data that only available in 20-21 season) into previous seasons. Null meaning that player doesnt
#exist in 20-21 season 
new_df['position'] = new_df.groupby('name')['position'].apply(lambda x: x.ffill().bfill())

In [7]:
clean_df = new_df[new_df['position'].notnull()]

In [8]:
#import team master
team_master = pd.read_csv('master_team_list.csv')

In [9]:
clean_df['id'] = clean_df['season'].astype(str) + '_' + clean_df['opponent_team'].astype(str)
team_master['id'] = team_master['season'].astype(str) + '_' + team_master['team'].astype(str)

C:\Users\harinn.hizhairi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
final_df = pd.merge(clean_df, team_master, on = 'id', how = 'left')

In [11]:
final_df = final_df.rename(columns={"team_name": "opp_team_name"})

In [12]:
final_df = final_df[['season_x', 'name', 'position', 'team_x', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'round', 'saves', 'selected', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'GW']]

In [13]:
final_df.to_csv('cleaned_merged_seasons.csv', encoding = 'utf-8')